# A Data Analysis on Just the Rides CSV File

Check the following questions:
1. incompleted rides
    -> requested, but canceled
    -> accepted, but canceled
    -> arrived at, but canceled
    -> picked up at, but canceled
    
2. completed rides
    -> all 5 from same ride id, -> mark rides completed, or not column

3. Time of ride (day) - earlymorning, morning, afternoon, evening, late-evening
    -> 
    
4. Time of ride (week)  - weekday, weekend, friday night?
    ->

3. Category type of ride - timely, versus long-distance
    -> ratio of distance to cost vs duration to the cost
##### A2. Ride category - time vs distance

Done on other df

In [85]:
# Necessary libs
import os
import numpy as np
import pandas as pd
import scipy
import sklearn
import datetime


# Plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import dabest

# custom libs
import sys
sys.path.append("../")
from src.data.rider import Fare, Ride

# Import magic commands for jupyter notebook 
# - autoreloading a module
# - profiling functions for memory usage and scripts
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Load in Data

In [77]:
# datadir = "../data/interim/"
# fpath = os.path.join(datadir, "driver_v1.csv")
                              
# df = pd.read_csv(fpath)

# print(os.listdir(datadir))

In [108]:
datadir = "../data/raw/"

ridetime_fpath = os.path.join(datadir, "ride_timestamps.csv")
                              
time_df = pd.read_csv(ridetime_fpath)

print(os.listdir(datadir))

['.gitkeep', 'ride_timestamps.csv', 'driver_ids.csv', 'ride_ids.csv']


In [109]:
display(time_df.head())

print("The size of the dataframes are: ", time_df.shape)

,ride_id,event,timestamp
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05


The size of the dataframes are:  (970405, 3)


In [110]:
# ensure datetime
time_df['timestamp'] = pd.to_datetime(time_df['timestamp'])

display(time_df.head())

,ride_id,event,timestamp
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05


##### A1. Incomplete/Complete ride

In [111]:
check_df = time_df.copy()
check_df.set_index("ride_id", drop=True, inplace=True)
check_df.drop(labels=["timestamp"], inplace=True, axis=1)
check_dict = check_df.to_dict('list')

display(check_df.head())
print(len(check_dict))

,event
ride_id,
00003037a262d9ee40e61b5c0718f7f0,requested_at
00003037a262d9ee40e61b5c0718f7f0,accepted_at
00003037a262d9ee40e61b5c0718f7f0,arrived_at
00003037a262d9ee40e61b5c0718f7f0,picked_up_at
00003037a262d9ee40e61b5c0718f7f0,dropped_off_at


1


In [179]:
# check each ride_id as "incompleted", or "completed"
complete_event = [
    "requested_at",
    "accepted_at",
    "arrived_at",
    "picked_up_at",
    "dropped_off_at",
]

complete_rides = {}
num_incomplete_rides = 0

for i, (_id, events) in enumerate(check_dict.items()):
    if not all(x in events for x in complete_event):
        complete_rides[_id] = 0
        num_incomplete_rides += 1
    else:
        complete_rides[_id] = 1

print("Number incomplete rides: ", num_incomplete_rides)
print("Total number of rides: ", len(ride_ids))

Number of unique rides:  194081
Number incomplete rides:  0
Total number of rides:  194081


In [127]:
display(time_df.head())

,ride_id,event,timestamp,hr_of_day
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19,2000-01-01 09:00:00
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51,2000-01-01 09:00:00
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31,2000-01-01 09:00:00
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33,2000-01-01 09:00:00
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05,2000-01-01 09:00:00


##### A2. Ride time of day

In [135]:
def compute_hr_day(df):
    x = df["timestamp"]
    hour=x.dt.time
    return hour

time_df["hr_of_day"] = time_df.pipe(compute_hr_day)
display(time_df.head())

,ride_id,event,timestamp,hr_of_day
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19,09:39:19
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51,09:39:51
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31,09:44:31
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33,09:44:33
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05,10:03:05


In [136]:
late_evening = ["22:00:00", "02:00:00"]
early_morning = ["02:00:00", "06:00:00"]
late_morning = ["06:00:00", "10:00:00"]
early_after = ["10:00:00", "14:00:00"]
late_after = ["14:00:00", "18:00:00"]
early_evening = ["18:00:00", "22:00:00"]

ranges = [early_morning, late_morning, early_after, late_after, early_evening, late_evening]
for i, tperiod in enumerate(ranges):
    ranges[i] = [datetime.datetime.strptime(x, '%H:%M:%S') for x in tperiod]
print(ranges)

[[datetime.datetime(1900, 1, 1, 2, 0), datetime.datetime(1900, 1, 1, 6, 0)], [datetime.datetime(1900, 1, 1, 6, 0), datetime.datetime(1900, 1, 1, 10, 0)], [datetime.datetime(1900, 1, 1, 10, 0), datetime.datetime(1900, 1, 1, 14, 0)], [datetime.datetime(1900, 1, 1, 14, 0), datetime.datetime(1900, 1, 1, 18, 0)], [datetime.datetime(1900, 1, 1, 18, 0), datetime.datetime(1900, 1, 1, 22, 0)], [datetime.datetime(1900, 1, 1, 22, 0), datetime.datetime(1900, 1, 1, 2, 0)]]


In [148]:
display(time_df.head())

,ride_id,event,timestamp,hr_of_day
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19,09:39:19
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51,09:39:51
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31,09:44:31
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33,09:44:33
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05,10:03:05


In [163]:
print(time_df.shape)
time_df=time_df.dropna()
print(time_df.shape)

(970405, 5)
(970404, 5)


In [165]:
def bin_f(x):
    if x.time() < datetime.time(6):
#         return "00:00-05:59"
        return 1
    elif x.time() < datetime.time(12):
#         return "06:00-11:59"
        return 2
    elif x.time() < datetime.time(18):
#         return "12:00-17:59"
        return 3
    else:
#         return "18:00-23:59"
        return 4

time_df["time_of_day"] = time_df["timestamp"].apply(bin_f)

display(time_df.head())
# grouped = df.groupby("Bin")
# grouped['X'].agg(np.std)

,ride_id,event,timestamp,hr_of_day,time_of_day
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19,09:39:19,2
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51,09:39:51,2
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31,09:44:31,2
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33,09:44:33,2
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05,10:03:05,2


In [167]:
# time_df["time_of_day"] = pd.cut(time_df.timestamp.dt.hour, bins=6)
time_df.drop("hr_of_day", inplace=True, axis=1)
display(time_df.head())

,ride_id,event,timestamp,time_of_day
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19,2
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51,2
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31,2
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33,2
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05,2


##### A3. Ride time of week

In [168]:
time_df['weekday'] = ((pd.DatetimeIndex(time_df.timestamp).dayofweek) // 5 == 1).astype(float)
display(time_df.head())

,ride_id,event,timestamp,time_of_day,weekday
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19,2,0.0
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51,2,0.0
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31,2,0.0
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33,2,0.0
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05,2,0.0


##### A4. Ride time of year

In [170]:
def compute_month(df):
    x = df["timestamp"]
    month=x.dt.month
    return month

time_df["month"] = time_df.pipe(compute_month)
display(time_df.head())

,ride_id,event,timestamp,time_of_day,weekday,month
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19,2,0.0,6
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51,2,0.0,6
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31,2,0.0,6
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33,2,0.0,6
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05,2,0.0,6


In [177]:
mindate = time_df["timestamp"].min()
maxdate = time_df["timestamp"].max()

print("The range of all rides for our data is: ", mindate, maxdate)

The range of all rides for our data is:  2016-03-28 05:48:18 2016-06-27 00:50:50


In [174]:
test = time_df.groupby(pd.Grouper(key="timestamp", freq='M'))

In [175]:
display(test.head())

,ride_id,event,timestamp,time_of_day,weekday,month
383080,655ccbeb2a62880159e20e986c1cdaeb,requested_at,2016-03-28 05:48:18,1,0.0,3
383081,655ccbeb2a62880159e20e986c1cdaeb,accepted_at,2016-03-28 05:48:23,1,0.0,3
383082,655ccbeb2a62880159e20e986c1cdaeb,arrived_at,2016-03-28 06:02:36,2,0.0,3
383083,655ccbeb2a62880159e20e986c1cdaeb,picked_up_at,2016-03-28 06:02:39,2,0.0,3
383084,655ccbeb2a62880159e20e986c1cdaeb,dropped_off_at,2016-03-28 06:37:51,2,0.0,3
4707,01380a4ad3b51c8ef791cdbf25bb48ac,arrived_at,2016-04-01 00:00:17,1,0.0,4
4708,01380a4ad3b51c8ef791cdbf25bb48ac,picked_up_at,2016-04-01 00:00:17,1,0.0,4
354105,5d81a42971402c8eeee0ca20b7146b33,requested_at,2016-04-01 00:05:26,1,0.0,4
354106,5d81a42971402c8eeee0ca20b7146b33,accepted_at,2016-04-01 00:05:33,1,0.0,4
354107,5d81a42971402c8eeee0ca20b7146b33,arrived_at,2016-04-01 00:08:15,1,0.0,4


# Save New Dataset on Rides

In [ ]:
fpath = "../data/interim/"

In [ ]:
time_df.to_csv(fpath)